In [6]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
sys.path.append('/users/mtaranov/genome3D/')
from models_3d import Genome3D_DNN_FC
from utils import concat_motifs_w_indx, normalize_features_sampl_by_f, concat_motifs, get_features, get_labels, subsample_data, normalize_features, reconstruct_2d, printMatrix, binarize, zscore, get_2D, plot_prediction, change_scale
#from sklearn.preprocessing import MinMaxScaler, StandardScaler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
path='/users/mtaranov/datasets_captureC_blood/'

X_train_node1 = path+'motifs/train_out_node1_3_0.0001/mat.npy'
X_train_node2 = path+'motifs/train_out_node2_3_0.0001/mat.npy'
X_train_window = path+'motifs/train_out_btw_nodes_3_0.0001/mat.npy'

X_test_node1 = path+'motifs/test_out_node1_3_0.0001/mat.npy'
X_test_node2 = path+'motifs/test_out_node2_3_0.0001/mat.npy'
X_test_window = path+'motifs/test_out_btw_nodes_3_0.0001/mat.npy'

X_valid_node1 = path+'motifs/valid_out_node1_3_0.0001/mat.npy'
X_valid_node2 = path+'motifs/valid_out_node2_3_0.0001/mat.npy'
X_valid_window = path+'motifs/valid_out_btw_nodes_3_0.0001/mat.npy'


In [ ]:
cell_types = ['Mon', 'Mac0', 'Mac1', 'Mac2', 'Neu', 'MK', 'EP', 'Ery', 'FoeT', 'nCD4', 'tCD4', 'aCD4', 'naCD4', 'nCD8', 'tCD8', 'nB', 'tB']

for cell_type in cell_types:
    y_train = get_labels(path+'labels/y_train_'+cell_type+'.npy')
    y_test = get_labels(path+'labels/y_test_'+cell_type+'.npy')
    y_valid = get_labels(path+'labels/y_valid_'+cell_type+'.npy')
    print 'labels'
    print y_train.shape,  y_valid.shape,  y_test.shape
    
    indx_train = np.load(path+'labels/y_train_'+cell_type+'_indx.npy')
    indx_test = np.load(path+'labels/y_test_'+cell_type+'_indx.npy')
    indx_valid = np.load(path+'labels/y_valid_'+cell_type+'_indx.npy')

    X_train=concat_motifs_w_indx([X_train_node1, X_train_node2, X_train_window], indx_train)
    X_valid=concat_motifs_w_indx([X_valid_node1, X_valid_node2, X_valid_window], indx_valid)
    X_test=concat_motifs_w_indx([X_test_node1, X_test_node2, X_test_window], indx_test)
    
    X_train_normalized, X_valid_normalized, X_test_normalized = normalize_features(X_train, X_valid, X_test)
    
    X_train_pairs = X_train_normalized.reshape(X_train_normalized.shape[0],X_train_normalized.shape[2]*X_train_normalized.shape[3])
    X_valid_pairs = X_valid_normalized.reshape(X_valid_normalized.shape[0],X_valid_normalized.shape[2]*X_valid_normalized.shape[3])
    X_test_pairs = X_test_normalized.reshape(X_test_normalized.shape[0],X_test_normalized.shape[2]*X_test_normalized.shape[3])

    print 'data'
    print X_train_pairs.shape, X_valid_pairs.shape, X_test_pairs.shape

    
    dnn_normalized = Genome3D_DNN_FC(num_features=5760, use_deep_CNN=True)
    validation_data = (X_valid_pairs, y_valid)
    dnn_normalized.train(X_train_pairs, y_train, validation_data)
    
    print 'PREDICTION IN', cell_type
    print(dnn_normalized.test(X_test_pairs, y_test))
    print(dnn_normalized.test(X_train_pairs, y_train))
    print(dnn_normalized.test(X_valid_pairs, y_valid))

labels
(11232, 1) (4589, 1) (2990, 1)
data
(11232, 5760) (4589, 5760) (2990, 5760)
Training model...
Train on 11232 samples, validate on 4589 samples
Epoch 1/100
11000/11232 [============================>.] - ETA: 0s - loss: 1.1950Epoch 0: validation loss: 11.441
Balanced Accuracy: 50.00%	auROC: 0.506	 auPRC: 0.564
Positve Accuracy: 0.00%	 Negative Accuracy: 100.00%
Recall at 5% | 10% | 25% | 50% FDR: 0.0% | 0.0% | 0.0% | 0.0%	Num Positives: 1214	 Num Negatives: 3375


11232/11232 [==============================] - 4s - loss: 1.1951 - val_loss: 11.4411
Epoch 2/100
11000/11232 [============================>.] - ETA: 0s - loss: 0.9553Epoch 1: validation loss: 3.104
Balanced Accuracy: 49.92%	auROC: 0.498	 auPRC: 0.263
Positve Accuracy: 0.00%	 Negative Accuracy: 99.84%
Recall at 5% | 10% | 25% | 50% FDR: 0.0% | 0.0% | 0.0% | 0.0%	Num Positives: 1214	 Num Negatives: 3375


11232/11232 [==============================] - 3s - loss: 0.9534 - val_loss: 3.1042
Epoch 3/100
11000/11232 [==========